# ASSIGNMENT 2

## Chin Wai Yee - Data Preprocessing

In [1]:
import os
import kagglehub
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Download latest version
path = kagglehub.dataset_download("tristanzhang32/ai-generated-images-vs-real-images")

ai_dir = path + "/train/fake"
real_dir = path + "/train/real"

# Label 0 = AI-generated, Label 1 = Real
ai_data = [(os.path.join(ai_dir, f), 0) for f in os.listdir(ai_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
real_data = [(os.path.join(real_dir, f), 1) for f in os.listdir(real_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

full_data = ai_data + real_data
labels = [label for _, label in full_data]

In [2]:
# Train/val/test split
train_val_data, test_data = train_test_split(full_data, test_size=0.15, stratify=labels, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.15, stratify=[label for _, label in train_val_data], random_state=42)

# Custom Dataset
class ImagePathDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        try:
            image = Image.open(img_path)
    
            if image.mode == 'P':
                image = image.convert('RGBA')
                background = Image.new("RGB", image.size, (255, 255, 255))
                image = Image.alpha_composite(background.convert('RGBA'), image).convert('RGB')
            else:
                image = image.convert('RGB')
    
            if self.transform:
                image = self.transform(image)
            return image, label
    
        except (OSError, IOError) as e:
            print(f"Skipping corrupted image: {img_path} ({str(e)})")
            return self.__getitem__((idx + 1) % len(self.data))  # Try next image

In [3]:
# Data Augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Standard transform for val/test
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [4]:
# Create datasets and loaders
train_dataset = ImagePathDataset(train_data, transform=train_transform)
val_dataset = ImagePathDataset(val_data, transform=test_transform)
test_dataset = ImagePathDataset(test_data, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## Loh Kin Ming - Model, Hyperparameters & Training

In [ ]:
import torch
from PIL import Image
import timm
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
model = model.to(device)

In [ ]:
## To Freeze Layers
## Uncomment below to run
# Freeze all layers
# for param in model.parameters():
#     param.requires_grad = False

# # Unfreeze only the classifier
# for param in model.classifier.parameters():
#     param.requires_grad = True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [ ]:
def train(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct, total = 0, 0
        
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        acc = correct / total * 100
        print(f"Epoch {epoch+1}: Loss={running_loss:.4f}, Accuracy={acc:.2f}%")

        # Evaluate on validation set
        evaluate(model, val_loader, "Validation")

def evaluate(model, loader, name="Test"):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"{name} Accuracy: {correct / total * 100:.2f}%")

In [ ]:
train(model, train_loader, val_loader, epochs=5)
evaluate(model, test_loader)

In [ ]:
# Save the full model
torch.save(model, "model_full.pth")

# Download the model file to local device
from IPython.display import FileLink

FileLink('model_full.pth')

## Brandon Ting En Junn - Evaluation & Demo

In [ ]:
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.models import efficientnet_b0
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_auc_score, roc_curve, cohen_kappa_score

In [ ]:
class ModelEvaluator():
    def __init__(self, model, data_loader):
        self.model = model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.y_true = []
        self.y_pred = []
        self.y_score = []

        self.model.eval()
        self.model.to(self.device)

        print("Running Evaluation...")
        with torch.no_grad():
            for inputs, labels in data_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)

                probits = torch.softmax(outputs, dim=1)

                _, pred = torch.max(probits, dim=1)

                self.y_true.extend(labels.to("cpu").numpy())
                self.y_pred.extend(pred.to("cpu").numpy())
                self.y_score.extend(probits[:, 1].to("cpu").numpy())
        print("Finished Evaluation...")

    def confusion_matrix(self):
        sns.heatmap(confusion_matrix(self.y_true, self.y_pred), annot=True)
        plt.title("Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.show()

    def accuracy(self):
        print(f"Accuracy: {accuracy_score(self.y_true, self.y_pred)}")

    def precision(self):
        print(f"Precision: {precision_score(self.y_true, self.y_pred)}")

    def recall(self):
        print(f"Recall: {recall_score(self.y_true, self.y_pred)}")

    def f1_score(self):
        print(f"F1-Score: {f1_score(self.y_true, self.y_pred)}")

    def kappa(self):
        print(f"Kappa Coefficient: {cohen_kappa_score(self.y_true, self.y_pred)}")

    def precision_recall_curve(self):
        precision, recall, _ = precision_recall_curve(self.y_true, self.y_score)
        plt.plot(recall, precision, marker='.')
        plt.title('Precision-Recall Curve')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.grid(True)
        plt.show()

    def auc_roc_curve(self):
        fpr, tpr, _ = roc_curve(self.y_true, self.y_score)
        auc_score = roc_auc_score(self.y_true, self.y_score)
        plt.title('AUC-ROC Curve')
        plt.xlabel('False Positive Rate (FPR)')
        plt.ylabel('True Positive Rate (TPR)')
        plt.plot(fpr, tpr, label=f'AUC = {auc_score:.4f}')
        plt.plot([0, 1], [0, 1], linestyle='--')
        plt.grid(True)
        plt.legend()
        plt.show()

    def summary(self):
        self.confusion_matrix()
        self.accuracy()
        self.precision()
        self.recall()
        self.f1_score()
        self.kappa()
        self.precision_recall_curve()
        self.auc_roc_curve()

In [ ]:
# model = efficientnet_b0()
# model.fc = torch.nn.Linear(model1.fc.in_features, 2)
# model.load_state_dict(torch.load('/kaggle/input/efficientnetb0/pytorch/default/1/model1_full.pth'))
model = torch.load("/kaggle/input/efficientnetb0/pytorch/default/1/model1_full.pth")
model_evaluator = ModelEvaluator(model=model, data_loader=test_loader)

In [ ]:
model_evaluator.summary()